In [65]:
import pandas as pd
import re


In [66]:
# Read Bronze layer data
df = pd.read_parquet('./Medallion_Architecture/bronze/Bronze.parquet')


Helper Functions

In [67]:
def fill_missing_values(df, columns_defaults: dict):
    for column, default_value in columns_defaults.items():
        df[column] = df[column].fillna(default_value)
    return df


In [68]:
def remove_punctuation(df, columns: list):
    for c in columns:
        df.loc[:,c] = df[c].str.replace(r'[^\w\s]|_', '', regex=True)
    return df
    

In [69]:
def convert_column_types(df, columns_types: dict):
    try:
        for column, dtype in columns_types.items():
            df[column] = df[column].astype(dtype)
        return df
    except Exception as e:
        print(f'{column} caused an issue')
        raise e


In [70]:
def check_formats(df, expected_formats: dict):
    incorrect_formats = []
    for column, datatype in df.dtypes.to_dict().items():
        expected_type = expected_formats.get(column)
        if expected_type != datatype:
            incorrect_formats.append((column, datatype, expected_type))

    incorrect_columns =[c[0] for c in incorrect_formats]
    correct_format_count = len([c for c in df.columns if c not in incorrect_columns])
    if incorrect_formats:
        print('Below are incorrect formats')
        print('-' * 50)
        print(f'Correct Column Count: {correct_format_count}')
        return pd.DataFrame(incorrect_formats, columns=['Column', 'Actual', 'Expected'])
    else:
        print('Validation Complete, no discrepancies')



In [71]:
def check_similarity(word1: str, word2: str) -> float:
    word_set1 = set(word1)
    word_set2 = set(word2)

    intersection = word_set1.intersection(word_set2)
    intersection_count = len(intersection)

    total_char_count = len(word_set1.union(word_set2))

    similarity = intersection_count / total_char_count
    return similarity


In [72]:
def check_misspelling(dataframe: pd.DataFrame, column: str, similarity_threshold: float) -> pd.DataFrame:
    all_unique_values = list(set(dataframe[column].tolist()))

    similarity_list = []
    for n in range(len(all_unique_values)):
        value1 = all_unique_values[n]
        for n2 in range(n + 1, len(all_unique_values)):
            value2 = all_unique_values[n2]
            similarity = round(check_similarity(value1, value2), 4)
            if similarity >= similarity_threshold:
                similarity_list.append([value1, value2, similarity])
    return pd.DataFrame(similarity_list, columns=['name1', 'name2', 'similarity'])


In [73]:
def titlecase(text):
    text = text.title()
    return re.sub(r"\'S\b", "'s", text)

Data Cleaning

In [74]:
df.head()

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,-34.0,0.0,Yes,No,1-Up Mushroom,64,26,4.26,Mario Tennis Aces,2025-10-27 10:50:47.281938
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,149.0,4.0,No,No,"Red Shell, Super Star",335,40,5.00,Mario Tennis Aces,2025-10-27 10:50:47.281938
2,Waluigi,None,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,174.0,1.0,No,Yes,Green Shell,182,57,5.50,Mario Kart 8 Deluxe,2025-10-27 10:50:47.281938
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,-1.0,5.0,No,Yes,1-Up Mushroom,333,84,6.00,Super Mario Bros.,2025-10-27 10:50:47.281938
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,28.0,2.0,Yes,No,"Red Shell, Banana Peel, Fire Flower",461,55,7.00,Super Mario World,2025-10-27 10:50:47.281938


In [75]:
df.isnull().sum()

Player Name                      9040
Team                             2960
World                               0
Vehicle Type                        0
Companion                           0
Kart Racing Rank                 9280
Platforming Rank                    0
Boss Battle Rank                    0
Power-Ups Used                      0
Kart Role                           0
Team Points                         0
Lives Lost                          0
Participation in Battle Mode        0
Mushroom Cup Participation      12400
Power-Ups Owned                     0
Coins Spent in Toad Town            0
Levels Completed                    0
Times Hit by Enemies                0
Primary Game                        0
loadDateTimeStamp                   0
dtype: int64

In [76]:
df.isnull().mean() * 100

Player Name                     18.833333
Team                             6.166667
World                            0.000000
Vehicle Type                     0.000000
Companion                        0.000000
Kart Racing Rank                19.333333
Platforming Rank                 0.000000
Boss Battle Rank                 0.000000
Power-Ups Used                   0.000000
Kart Role                        0.000000
Team Points                      0.000000
Lives Lost                       0.000000
Participation in Battle Mode     0.000000
Mushroom Cup Participation      25.833333
Power-Ups Owned                  0.000000
Coins Spent in Toad Town         0.000000
Levels Completed                 0.000000
Times Hit by Enemies             0.000000
Primary Game                     0.000000
loadDateTimeStamp                0.000000
dtype: float64

In [77]:
fill_missing_values(df, {'Player Name': 'Unknown Player', 'Team': 'Unknown Team', 'Kart Racing Rank': 'Unknown Rank', 'Mushroom Cup Participation': 'Unknown'})


,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,-34.0,0.000000,Yes,No,1-Up Mushroom,64,26,4.26,Mario Tennis Aces,2025-10-27 10:50:47.281938
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,149.0,4.000000,No,No,"Red Shell, Super Star",335,40,5.00,Mario Tennis Aces,2025-10-27 10:50:47.281938
2,Waluigi,Unknown Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,174.0,1.000000,No,Yes,Green Shell,182,57,5.50,Mario Kart 8 Deluxe,2025-10-27 10:50:47.281938
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,-1.0,5.000000,No,Yes,1-Up Mushroom,333,84,6.00,Super Mario Bros.,2025-10-27 10:50:47.281938
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,28.0,2.000000,Yes,No,"Red Shell, Banana Peel, Fire Flower",461,55,7.00,Super Mario World,2025-10-27 10:50:47.281938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,Toadette,Green Caps,Star World,Mach 8,Luma,B,A,S,8,Drifter,-84.0,0.000000,No,Yes,"1-Up Mushroom, Super Mushroom",172,43,5.00,Super Mario Bros.,2025-10-27 10:50:47.567800
47996,Bowser Jr.,Dino Buddies,Donut Plains,Circuit Special,Koopa Troopa,C,C,B,13,Speedster,110.0,1.083565,No,Unknown,Fire Flower,11,72,9.00,Super Mario World,2025-10-27 10:50:47.567800
47997,Luigi,Green Caps,dONUT pLAINS,Biddybuggy,Shy Guy,A,D,D,27,Drifter,-38.0,4.000000,Yes,Unknown,Super Star,460,25,2.00,Super Mario Odyssey,2025-10-27 10:50:47.567800
47998,Toadette,Koopa Clan,mUSHROOM kINGDOM,Circuit Special,Yoshi,B,A,B,8,Item Specialist,111.0,0.000000,Yes,Yes,"Super Star, Banana Peel, Green Shell",82,39,4.00,Mario Kart 8 Deluxe,2025-10-27 10:50:47.567800


In [78]:
df.isnull().mean() * 100

Player Name                     0.0
Team                            0.0
World                           0.0
Vehicle Type                    0.0
Companion                       0.0
Kart Racing Rank                0.0
Platforming Rank                0.0
Boss Battle Rank                0.0
Power-Ups Used                  0.0
Kart Role                       0.0
Team Points                     0.0
Lives Lost                      0.0
Participation in Battle Mode    0.0
Mushroom Cup Participation      0.0
Power-Ups Owned                 0.0
Coins Spent in Toad Town        0.0
Levels Completed                0.0
Times Hit by Enemies            0.0
Primary Game                    0.0
loadDateTimeStamp               0.0
dtype: float64

In [79]:
remove_punctuation(df, ['Kart Role'])

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,-34.0,0.000000,Yes,No,1-Up Mushroom,64,26,4.26,Mario Tennis Aces,2025-10-27 10:50:47.281938
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,149.0,4.000000,No,No,"Red Shell, Super Star",335,40,5.00,Mario Tennis Aces,2025-10-27 10:50:47.281938
2,Waluigi,Unknown Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,174.0,1.000000,No,Yes,Green Shell,182,57,5.50,Mario Kart 8 Deluxe,2025-10-27 10:50:47.281938
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,-1.0,5.000000,No,Yes,1-Up Mushroom,333,84,6.00,Super Mario Bros.,2025-10-27 10:50:47.281938
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,28.0,2.000000,Yes,No,"Red Shell, Banana Peel, Fire Flower",461,55,7.00,Super Mario World,2025-10-27 10:50:47.281938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,Toadette,Green Caps,Star World,Mach 8,Luma,B,A,S,8,Drifter,-84.0,0.000000,No,Yes,"1-Up Mushroom, Super Mushroom",172,43,5.00,Super Mario Bros.,2025-10-27 10:50:47.567800
47996,Bowser Jr.,Dino Buddies,Donut Plains,Circuit Special,Koopa Troopa,C,C,B,13,Speedster,110.0,1.083565,No,Unknown,Fire Flower,11,72,9.00,Super Mario World,2025-10-27 10:50:47.567800
47997,Luigi,Green Caps,dONUT pLAINS,Biddybuggy,Shy Guy,A,D,D,27,Drifter,-38.0,4.000000,Yes,Unknown,Super Star,460,25,2.00,Super Mario Odyssey,2025-10-27 10:50:47.567800
47998,Toadette,Koopa Clan,mUSHROOM kINGDOM,Circuit Special,Yoshi,B,A,B,8,Item Specialist,111.0,0.000000,Yes,Yes,"Super Star, Banana Peel, Green Shell",82,39,4.00,Mario Kart 8 Deluxe,2025-10-27 10:50:47.567800


In [80]:
check_misspelling(df, 'Player Name', 0.8)

,name1,name2,similarity
0,Mqario,MaMrio,0.8333
1,Mqario,Maraio,0.8333
2,Mqario,Mario,0.8333
3,Mqario,Mariio,0.8333
4,Mqario,Maorio,0.8333
...,...,...,...
21346,Bowser JUr.,Bkowser Jr.,0.8182
21347,Bowser JUr.,Bowser Jr.P,0.8182
21348,BowserB Jr.,Bkowser Jr.,0.9000
21349,BowserB Jr.,Bowser Jr.P,0.9000


In [81]:
expected_formats = {
    'Player Name': 'string',
    'Kart Racing Rank': 'string',
    'Kart Role': 'string',
    'Mushroom Cup Participation': 'bool',
    'Participation in Battle Mode': 'bool',
    'Platforming Rank': 'string',
    'Boss Battle Rank': 'string',
    'Power-Ups Owned': 'string',
    'Power-Ups Used': 'int32',
    'Team Points': 'int32',
    'Levels Completed': 'int32',
    'Lives Lost': 'int32',
    'Times Hit by Enemies': 'int32',
    'Vehicle Type': 'string',
    'World': 'string',
    'Coins Spent in Toad Town': 'int32',
    'Companion': 'string',
    'Primary Game': 'string',
    'loadDateTimeStamp': 'datetime64[ns]',
    'Team': 'string'
}

In [82]:
check_formats(df, expected_formats=expected_formats)

Below are incorrect formats
--------------------------------------------------
Correct Column Count: 0


,Column,Actual,Expected
0,Player Name,object,string
1,Team,object,string
2,World,object,string
3,Vehicle Type,object,string
4,Companion,object,string
5,Kart Racing Rank,object,string
6,Platforming Rank,object,string
7,Boss Battle Rank,object,string
8,Power-Ups Used,int64,int32
9,Kart Role,object,string


In [83]:
convert_column_types(df, columns_types=expected_formats)

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,-34,0,True,True,1-Up Mushroom,64,26,4,Mario Tennis Aces,2025-10-27 10:50:47.281938
1,PeachK,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,149,4,True,True,"Red Shell, Super Star",335,40,5,Mario Tennis Aces,2025-10-27 10:50:47.281938
2,Waluigi,Unknown Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,174,1,True,True,Green Shell,182,57,5,Mario Kart 8 Deluxe,2025-10-27 10:50:47.281938
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,-1,5,True,True,1-Up Mushroom,333,84,6,Super Mario Bros.,2025-10-27 10:50:47.281938
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,28,2,True,True,"Red Shell, Banana Peel, Fire Flower",461,55,7,Super Mario World,2025-10-27 10:50:47.281938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,Toadette,Green Caps,Star World,Mach 8,Luma,B,A,S,8,Drifter,-84,0,True,True,"1-Up Mushroom, Super Mushroom",172,43,5,Super Mario Bros.,2025-10-27 10:50:47.567800
47996,Bowser Jr.,Dino Buddies,Donut Plains,Circuit Special,Koopa Troopa,C,C,B,13,Speedster,110,1,True,True,Fire Flower,11,72,9,Super Mario World,2025-10-27 10:50:47.567800
47997,Luigi,Green Caps,dONUT pLAINS,Biddybuggy,Shy Guy,A,D,D,27,Drifter,-38,4,True,True,Super Star,460,25,2,Super Mario Odyssey,2025-10-27 10:50:47.567800
47998,Toadette,Koopa Clan,mUSHROOM kINGDOM,Circuit Special,Yoshi,B,A,B,8,Item Specialist,111,0,True,True,"Super Star, Banana Peel, Green Shell",82,39,4,Mario Kart 8 Deluxe,2025-10-27 10:50:47.567800


In [84]:
check_formats(df, expected_formats=expected_formats)

Validation Complete, no discrepancies


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Player Name                   48000 non-null  string        
 1   Team                          48000 non-null  string        
 2   World                         48000 non-null  string        
 3   Vehicle Type                  48000 non-null  string        
 4   Companion                     48000 non-null  string        
 5   Kart Racing Rank              48000 non-null  string        
 6   Platforming Rank              48000 non-null  string        
 7   Boss Battle Rank              48000 non-null  string        
 8   Power-Ups Used                48000 non-null  int32         
 9   Kart Role                     48000 non-null  string        
 10  Team Points                   48000 non-null  int32         
 11  Lives Lost                  

In [86]:
# Checking for possible misspellings of "Mario"
mario_names = df['Player Name'].dropna().unique()
mario_names = [name for name in mario_names if 'mario' in name.lower()]

In [87]:
possible_misspellings = check_misspelling(
    pd.DataFrame({'Player Name': mario_names}),
    column='Player Name',
    similarity_threshold=0.8
)

print(possible_misspellings)

     name1   name2  similarity
0   Mariot  MMario      0.8333
1   Mariot   Mario      0.8333
2   MarioX  MMario      0.8333
3   MarioX   Mario      0.8333
4   Mariod  MMario      0.8333
5   Mariod   Mario      0.8333
6   MarioU  MMario      0.8333
7   MarioU   Mario      0.8333
8   Marioz  MMario      0.8333
9   Marioz   Mario      0.8333
10  Marioy  MMario      0.8333
11  Marioy   Mario      0.8333
12  MarioS  MMario      0.8333
13  MarioS   Mario      0.8333
14  Marioe  MMario      0.8333
15  Marioe   Mario      0.8333
16  MMario  Mariof      0.8333
17  MMario  MarioA      0.8333
18  MMario  Mariop      0.8333
19  MMario   Mario      1.0000
20  MMario  MarioC      0.8333
21  MMario  Mariol      0.8333
22  MMario  Mariom      0.8333
23  MMario  Mariou      0.8333
24  MMario  MarioN      0.8333
25  MMario  MarioZ      0.8333
26  MMario  MarioV      0.8333
27  MMario  Mariob      0.8333
28  Mariof   Mario      0.8333
29  MarioA   Mario      0.8333
30  Mariop   Mario      0.8333
31   Mar

In [88]:
# Standardize the values in the Player Name column
valid_player_names = ["Mario", "Luigi", "Peach", "Daisy", "Yoshi", "Toad", "Toadette", "Rosalina", "Wario", "Waluigi", "Bowser", "Bowser Jr."]

similar_df = check_misspelling(df, 'Player Name', 0.6)

for i, row in df.iterrows():
    player = row['Player Name']
    best_match = None
    best_similarity = 0

    for valid in valid_player_names:
        similarity = check_similarity(player, valid)
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = valid

    if best_similarity >= 0.6:
        df.at[i, 'Player Name'] = best_match

In [89]:
print(df['Player Name'].unique())

<StringArray>
[         'Yoshi',          'Peach',        'Waluigi',     'Bowser Jr.',
       'Toadette',          'Mario',         'Bowser',           'Toad',
          'Daisy',       'Rosalina',          'Luigi',          'Wario',
 'Unknown Player']
Length: 13, dtype: string


In [90]:
# Remove leading and trailing spaces from Vehicle Type, World, and Primary Game
cols = ['Vehicle Type', 'World', 'Primary Game']

for col in cols:
    df[col] = df[col].str.strip()

In [91]:
df.head()

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,pOLTERPUP,A,A,A,12,Drifter,-34,0,True,True,1-Up Mushroom,64,26,4,Mario Tennis Aces,2025-10-27 10:50:47.281938
1,Peach,GREEN CAPS,Donut Plains,Circuit Special,kOOPA tROOPA,C,A,B,16,Drifter,149,4,True,True,"Red Shell, Super Star",335,40,5,Mario Tennis Aces,2025-10-27 10:50:47.281938
2,Waluigi,Unknown Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,174,1,True,True,Green Shell,182,57,5,Mario Kart 8 Deluxe,2025-10-27 10:50:47.281938
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,-1,5,True,True,1-Up Mushroom,333,84,6,Super Mario Bros.,2025-10-27 10:50:47.281938
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,tOAD,C,C,B,10,Blocker,28,2,True,True,"Red Shell, Banana Peel, Fire Flower",461,55,7,Super Mario World,2025-10-27 10:50:47.281938


In [92]:
# Titlecase for Team, Companion, and World
title_columns = ['Team', 'Companion', 'World']

for col in title_columns:
    df[col] = df[col].apply(titlecase)

In [93]:
df.head()

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,Polterpup,A,A,A,12,Drifter,-34,0,True,True,1-Up Mushroom,64,26,4,Mario Tennis Aces,2025-10-27 10:50:47.281938
1,Peach,Green Caps,Donut Plains,Circuit Special,Koopa Troopa,C,A,B,16,Drifter,149,4,True,True,"Red Shell, Super Star",335,40,5,Mario Tennis Aces,2025-10-27 10:50:47.281938
2,Waluigi,Unknown Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,174,1,True,True,Green Shell,182,57,5,Mario Kart 8 Deluxe,2025-10-27 10:50:47.281938
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,-1,5,True,True,1-Up Mushroom,333,84,6,Super Mario Bros.,2025-10-27 10:50:47.281938
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,Toad,C,C,B,10,Blocker,28,2,True,True,"Red Shell, Banana Peel, Fire Flower",461,55,7,Super Mario World,2025-10-27 10:50:47.281938


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Player Name                   48000 non-null  string        
 1   Team                          48000 non-null  object        
 2   World                         48000 non-null  object        
 3   Vehicle Type                  48000 non-null  string        
 4   Companion                     48000 non-null  object        
 5   Kart Racing Rank              48000 non-null  string        
 6   Platforming Rank              48000 non-null  string        
 7   Boss Battle Rank              48000 non-null  string        
 8   Power-Ups Used                48000 non-null  int32         
 9   Kart Role                     48000 non-null  string        
 10  Team Points                   48000 non-null  int32         
 11  Lives Lost                  

In [95]:
df.to_parquet('./Medallion_Architecture/silver/Silver.parquet', index=False)